In [15]:
import csv
import codecs
import subprocess
import os
import re
import glob

# get the p-value from the original diwan sheet to be attached to the 236 names
pval_by_name = dict()
#with codecs.open("diwan_missing_archaea.csv", "r",encoding='utf-8', errors='ignore') as f:
with codecs.open("diwan_missing_bacteria.csv", "r",encoding='utf-8', errors='ignore') as f:
    for row in f:
        if (row.startswith("Species")):
            continue
        data = row.strip().split(",")
        name = data[0].strip()
        p_val = data[14]
        #print (name + " " + str(p_val))
        species_dict_by_name[name] = p_val

#with codecs.open("Diwan_MissingArchaea_FilesInfo.csv", "r",encoding='utf-8', errors='ignore') as f:
 #   with open('Diwan_MissingArchaea_FilesInfo_withPvalshuffle.csv', 'w') as fout:
with codecs.open("Diwan_MissingBacteria_FilesInfo.csv", "r",encoding='utf-8', errors='ignore') as f:
    with open('Diwan_MissingBacteria_FilesInfo_withPvalshuffle.csv', 'w') as fout:
        csvWriter = csv.writer(fout, delimiter=',')
        csvWriter.writerow(["name","full_file_prefix", "16S_end", "Diwan_SD", "pvalshuff (aff5 to highest)"])
        for row in f:
            if (row.startswith("name")):
                continue
            data = row.strip().split(",")
            name = data[0].strip()
            data.append(species_dict_by_name[name])
            #print(data)
            csvWriter.writerow(data)

In [13]:
import csv
import codecs
import subprocess
import os
import re
import glob

# get the p-value from the original diwan sheet to be attached to the 236 names
pval_by_name = dict()
with codecs.open("DiwanSupplementOriginal.csv", "r",encoding='utf-8', errors='ignore') as f:
    for row in f:
        if (row.startswith("Species")):
            continue
        data = row.strip().split(",")
        name = data[0].strip()
        p_val = data[14]
        #print (name + " " + str(p_val))
        species_dict_by_name[name] = p_val
        
with codecs.open("DiwanDataFilesInfo.csv", "r",encoding='utf-8', errors='ignore') as f:
    with open('DiwanDataFilesInfo_withPvalshuffle.csv', 'w') as fout:
        csvWriter = csv.writer(fout, delimiter=',')
        csvWriter.writerow(["name","full_file_prefix", "16S_end", "Diwan_SD", "pvalshuff (aff5 to highest)"])
        for row in f:
            if (row.startswith("name")):
                continue
            data = row.strip().split(",")
            name = data[0].strip()
            data.append(species_dict_by_name[name])
            #print(data)
            csvWriter.writerow(data)

In [17]:
import csv
import codecs
import subprocess
import os
import re
import glob

# process the data file to run the india paper stuff - just run the processing on the 48 files not found in bacteria
# on the previous run

species_names = []
species_dict_by_name = dict()
with codecs.open("diwan_missing_archaea.csv", "r",encoding='utf-8', errors='ignore') as f:
    for row in f:
        if (row.startswith("Species")):
            continue
        data = row.strip().split(",")
        name = data[0].strip()
        diwan_anti_sd = data[8][1:-1].upper()
        diwan_anti_sd = diwan_anti_sd.replace('U', 'T')
        revcompl = lambda x: ''.join([{'A':'T','C':'G','G':'C','T':'A'}[B] for B in x][::-1])
        diwan_sd = revcompl(diwan_anti_sd)
        #if (diwan_sd != "AGGAGG"):
         #   print(name, " ", diwan_sd)
        species_names.append(name)
        species_dict_by_name[name] = dict()
        species_dict_by_name[name]["diwan_sd"] = diwan_sd
        species_dict_by_name[name]["found_data_files"] = False
        species_dict_by_name[name]["assebly_file_prefix"] = ""
        species_dict_by_name[name]["16S_end"] = ""
        species_dict_by_name[name]["original_row"] = row
        
        
# read the large excel file, and find the correct genome
with codecs.open("/scratch4/moamin/ncbi_archaea_db/archaea_helix45_16s18_organism_information.csv", "r",encoding='utf-8', errors='ignore') as f:
    for row in f:
        data = row.strip().split(",")
        name = data[3].strip()
        if ((name in species_dict_by_name) and ("Complete Genome" in data[1].strip())):
            species_dict_by_name[name]["16S_end"] = data[23]
            prefix = data[0]
            species_dict_by_name[name]["assebly_file_prefix"] = prefix
            species_dict_by_name[name]["found_data_files"] = True

count = 0
with open('Diwan_MissingArchaea_FilesInfo.csv', 'w') as f:
    csvWriter = csv.writer(f, delimiter=',')
    csvWriter.writerow(["name","full_file_prefix", "16S_end", "Diwan_SD"])
    # count missing names, and get the full file path, output everything to a csv file
    for name in species_names:
        if (species_dict_by_name[name]["found_data_files"] == False):
            #print("Not Found for " + name)
            print(species_dict_by_name[name]["original_row"])
            count += 1
        else:
            prefix = species_dict_by_name[name]["assebly_file_prefix"]
            #files = [f for f in os.listdir('/scratch4/moamin/ncbi_bacteria_db/GbBac_GFF/') if re.match(r'*.gz', f)]
            names= glob.glob('/scratch4/moamin/ncbi_archaea_db/GbArchaea_GFF/' + prefix + '*.gz')
            if (len(names) > 1):
                print("Error, more than 1 file with prefix:", first)
            else:
                full_prefix = os.path.split(names[0])[1]
                full_prefix = re.sub('_genomic\.gff\.gz$', '', full_prefix)
                #print(full_prefix)
                csvWriter.writerow([name, full_prefix, species_dict_by_name[name]["16S_end"], species_dict_by_name[name]["diwan_sd"]])
        
# getting 236 out of 285 species (found no matches for 48 species)
print("Not found matches for ", count, " species")

Not found matches for  0  species


In [20]:
import csv
import codecs
import subprocess
import os
import re
import glob

# process the data file to run the india paper stuff

species_names = []
species_dict_by_name = dict()
with codecs.open("diwan_missing_bacteria.csv", "r",encoding='utf-8', errors='ignore') as f:
#with codecs.open("DiwanSupplementOriginal.csv", "r",encoding='utf-8', errors='ignore') as f:
    for row in f:
        if (row.startswith("Species")):
            continue
        data = row.strip().split(",")
        name = data[0].strip()
        diwan_anti_sd = data[8][1:-1].upper()
        diwan_anti_sd = diwan_anti_sd.replace('U', 'T')
        revcompl = lambda x: ''.join([{'A':'T','C':'G','G':'C','T':'A'}[B] for B in x][::-1])
        diwan_sd = revcompl(diwan_anti_sd)
        #if (diwan_sd != "AGGAGG"):
         #   print(name, " ", diwan_sd)
        species_names.append(name)
        species_dict_by_name[name] = dict()
        species_dict_by_name[name]["diwan_sd"] = diwan_sd
        species_dict_by_name[name]["found_data_files"] = False
        species_dict_by_name[name]["assebly_file_prefix"] = ""
        species_dict_by_name[name]["16S_end"] = ""
        species_dict_by_name[name]["original_row"] = row
        
        
# read the large excel file, and find the correct genome
with codecs.open("/scratch4/moamin/ncbi_bacteria_db/general_helix45_16s18_organism_information.csv", "r",encoding='utf-8', errors='ignore') as f:
    for row in f:
        data = row.strip().split(",")
        name = data[3].strip()
        #if (name in species_dict_by_name):
        if ((name in species_dict_by_name) and ("Complete Genome" in data[1].strip())):
            species_dict_by_name[name]["16S_end"] = data[23]
            prefix = data[0]
            species_dict_by_name[name]["assebly_file_prefix"] = prefix
            species_dict_by_name[name]["found_data_files"] = True

count = 0
with open('Diwan_MissingBacteria_FilesInfo.csv', 'w') as f:
#with open('DiwanDataFilesInfo.csv', 'w') as f:
    csvWriter = csv.writer(f, delimiter=',')
    csvWriter.writerow(["name","full_file_prefix", "16S_end", "Diwan_SD"])
    # count missing names, and get the full file path, output everything to a csv file
    for name in species_names:
        if (species_dict_by_name[name]["found_data_files"] == False):
            #print("Not Found for " + name)
            print(species_dict_by_name[name]["original_row"])
            count += 1
        else:
            prefix = species_dict_by_name[name]["assebly_file_prefix"]
            #files = [f for f in os.listdir('/scratch4/moamin/ncbi_bacteria_db/GbBac_GFF/') if re.match(r'*.gz', f)]
            names= glob.glob('/scratch4/moamin/ncbi_bacteria_db/GbBac_GFF/' + prefix + '*.gz')
            if (len(names) > 1):
                print("Error, more than 1 file with prefix:", first)
            else:
                full_prefix = os.path.split(names[0])[1]
                full_prefix = re.sub('_genomic\.gff\.gz$', '', full_prefix)
                #print(full_prefix)
                csvWriter.writerow([name, full_prefix, species_dict_by_name[name]["16S_end"], species_dict_by_name[name]["diwan_sd"]])
        
# getting 236 out of 285 species (found no matches for 48 species)
print("Not found matches for ", count, " species")

Not found matches for  0  species


In [10]:
import csv
import codecs
import subprocess
import os
import re
import glob


# process the data file to run pipeline on Ruhul's 153 files
with open('Ruhul_Annotate_153_DataFilesInfo.csv', 'w') as f:
    csvWriter = csv.writer(f, delimiter=',')
    csvWriter.writerow(["name","full_file_prefix", "16S_end", "Diwan_SD"])
    # read the large excel file, and find the correct genome
    with codecs.open("ruhul_re-annotation_all_organism_information.csv", "r",encoding='utf-8', errors='ignore') as f:
        for row in f:
            data = row.strip().split(",")
            name = data[3].strip()
            prefix = data[0]
            end_of_16S = data[23]
            names= glob.glob('/scratch4/moamin/ncbi_bacteria_db/GbBac_GFF/' + prefix + '*.gz')
            if (len(names) > 1):
                print("Error, more than 1 file with prefix:", first)
            else:
                full_prefix = os.path.split(names[0])[1]
                full_prefix = re.sub('_genomic\.gff\.gz$', '', full_prefix)
                #print(full_prefix)
                csvWriter.writerow([name, full_prefix, end_of_16S, ""])       
             

In [23]:
import csv
import codecs
import subprocess
import os
import re
import glob


# process the data file to run pipeline on original files from the paper

#with open('Our_NOT_IN_DIWAN_SD_Bacteria_DataFilesInfo.csv', 'w') as f:
with open('Our_NOT_IN_DIWAN_SD_Archaea_DataFilesInfo.csv', 'w') as f:
    csvWriter = csv.writer(f, delimiter=',')
    csvWriter.writerow(["name","full_file_prefix", "16S_end", "Diwan_SD"])
    # read the large excel file, and find the correct genome
    #with codecs.open("our_from_paper_not_in_diwan_bacteria.csv", "r",encoding='utf-8', errors='ignore') as f:
    with codecs.open("our_from_paper_not_in_diwan_archaea.csv", "r",encoding='utf-8', errors='ignore') as f:
        for row in f:
            data = row.strip().split(",")
            name = data[3].strip()
            prefix = data[0]
            end_of_16S = data[23]
            #names= glob.glob('/scratch4/moamin/ncbi_bacteria_db/GbBac_GFF/' + prefix + '*.gz')
            names= glob.glob('/scratch4/moamin/ncbi_archaea_db/GbArchaea_GFF/' + prefix + '*.gz')
            if (len(names) > 1):
                print("Error, more than 1 file with prefix:", first)
            else:
                print(names)
                print(prefix)
                full_prefix = os.path.split(names[0])[1]
                full_prefix = re.sub('_genomic\.gff\.gz$', '', full_prefix)
                #print(full_prefix)
                csvWriter.writerow([name, full_prefix, end_of_16S, ""])       
             



['/scratch4/moamin/ncbi_archaea_db/GbArchaea_GFF/GCA_000191585.1_ASM19158v1_genomic.gff.gz']
GCA_000191585.1
['/scratch4/moamin/ncbi_archaea_db/GbArchaea_GFF/GCA_000306765.2_ASM30676v2_genomic.gff.gz']
GCA_000306765.2
['/scratch4/moamin/ncbi_archaea_db/GbArchaea_GFF/GCA_000017185.1_ASM1718v1_genomic.gff.gz']
GCA_000017185.1


In [23]:
# post-processing - is SD that is found in the tail?
import csv
import codecs
import subprocess
import os
import re
import glob

# process the data file to run pipeline on our stuff


#with open('DiwanDataFiles_WithAllStats_236_AGGAGG_and_Tompa_v4_rounded_annotated.csv', 'w') as f:
#with open('Ruhul_Annotate_153_DataFiles_WithAllStats_only_tompa_v4_153_rounded_annotated.csv', 'w') as f:
with open('Diwan_Missing_48_allstats_rounded_annotated.csv', 'w') as f:
    csvWriter = csv.writer(f, delimiter=',')
    # read the large excel file, and find the correct genome
    with codecs.open("Diwan_Missing_48_allstats.csv", "r",encoding='utf-8', errors='ignore') as f:
    #with codecs.open("Ruhul_Annotate_153_DataFiles_WithAllStats_only_tompa_v4_153.csv", "r",encoding='utf-8', errors='ignore') as f: 
    #with codecs.open("DiwanDataFiles_WithAllStats_236_AGGAGG_and_Tompa_v4_rounded.csv", "r",encoding='utf-8', errors='ignore') as f:
        for row in f:
            data = row.strip().split(",")
            if (data[0] == "name"):
                csvWriter.writerow(data + ["found in 16S"])
                continue
            seq = data[3]
            complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
            reverse_complement = "".join(complement.get(base, base) for base in reversed(seq))
            #print(reverse_complement)
            if (("-" not in seq) and ("-" not in data[2]) and (reverse_complement in data[2])):
                csvWriter.writerow(data + ["yes"])
            else:
                csvWriter.writerow(data + ["no"])
   